# Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Topic Modeling - Attempt #1 (All Text)

In [ ]:
!pip list

Package                               Version
------------------------------------- ------------------
absl-py                               1.4.0
accelerate                            1.5.2
aiohappyeyeballs                      2.6.1
aiohttp                               3.11.15
aiosignal                             1.3.2
alabaster                             1.0.0
albucore                              0.0.23
albumentations                        2.0.5
ale-py                                0.10.2
altair                                5.5.0
annotated-types                       0.7.0
anyio                                 4.9.0
argon2-cffi                           23.1.0
argon2-cffi-bindings                  21.2.0
array_record                          0.7.1
arviz                                 0.21.0
astropy                               7.0.1
astropy-iers-data                     0.2025.4.7.0.35.30
astunparse                            1.6.3
atpublic                              5.1

In [ ]:
!pip uninstall -y numpy

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2


In [ ]:
!pip install numpy==1.24.4 gensim==4.3.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.0 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
j

# Topic Modeling - Attempt #1

In [ ]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,aaaaah,aaah,aah,aback,abandoned,abbott,abc,abcs,ability,abject,...,zipup,zombie,zombies,zone,zoneyoure,zoning,zoo,zookeeper,zoom,zuckerberg
adam,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
ali,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,1,0,0,0,0,0,0,1,0,0,...,0,1,1,0,0,1,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chad,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ellen,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
gabriel,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,adam,ali,anthony,bill,bo,chad,dave,ellen,gabriel,hasan,...,jim,joe,john,louis,michelle,mike,nate,ricky,ronny,theo
aaaaah,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaah,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,0
aback,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abandoned,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [ ]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [ ]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.005*"theyre" + 0.004*"okay" + 0.004*"look" + 0.004*"good" + 0.004*"say" + 0.004*"going" + 0.004*"theres" + 0.003*"want" + 0.003*"cause" + 0.003*"hes"'),
 (1,
  '0.007*"fucking" + 0.006*"shit" + 0.005*"cause" + 0.005*"man" + 0.005*"say" + 0.005*"hes" + 0.005*"didnt" + 0.005*"little" + 0.005*"oh" + 0.004*"goes"')]

In [ ]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.009*"fucking" + 0.007*"shit" + 0.006*"man" + 0.006*"cause" + 0.006*"goes" + 0.006*"hes" + 0.005*"going" + 0.005*"little" + 0.005*"fuck" + 0.005*"theyre"'),
 (1,
  '0.006*"okay" + 0.005*"good" + 0.005*"want" + 0.004*"look" + 0.004*"hes" + 0.004*"didnt" + 0.004*"theyre" + 0.004*"uh" + 0.004*"love" + 0.004*"christmas"'),
 (2,
  '0.005*"say" + 0.005*"didnt" + 0.005*"really" + 0.005*"cause" + 0.004*"theyre" + 0.004*"shit" + 0.004*"good" + 0.004*"mean" + 0.004*"oh" + 0.004*"way"')]

In [ ]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.006*"theyre" + 0.006*"say" + 0.005*"cause" + 0.005*"didnt" + 0.004*"okay" + 0.004*"good" + 0.004*"want" + 0.004*"oh" + 0.004*"thing" + 0.004*"theres"'),
 (1,
  '0.005*"didnt" + 0.004*"christmas" + 0.004*"say" + 0.004*"did" + 0.004*"really" + 0.004*"good" + 0.004*"little" + 0.004*"mean" + 0.003*"way" + 0.003*"oh"'),
 (2,
  '0.000*"good" + 0.000*"hes" + 0.000*"fucking" + 0.000*"theyre" + 0.000*"say" + 0.000*"really" + 0.000*"cause" + 0.000*"want" + 0.000*"didnt" + 0.000*"going"'),
 (3,
  '0.009*"fucking" + 0.007*"shit" + 0.006*"hes" + 0.006*"cause" + 0.006*"goes" + 0.006*"man" + 0.005*"going" + 0.005*"fuck" + 0.005*"little" + 0.005*"want"')]

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [ ]:
import nltk

# Download the necessary data for the PerceptronTagger, including the specific language model
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt') # Download the 'punkt' resource for word_tokenize
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger_eng') # Download the English language model specifically
nltk.download('punkt_tab') # Download the 'punkt_tab' resource for sentence tokenization

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)]
    return ' '.join(all_nouns)

In [ ]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcript
adam,adam sandler love you transcript scraps from t...
ali,ali wong baby cobra transcript scraps from the...
anthony,anthony jeselnik thoughts and prayers full tra...
bill,bill burr im sorry you feel that way full tran...
bo,scraps from the loft skip to content search mo...
chad,chad daniels dad chaniels transcript scraps fr...
dave,dave chappelle the age of spin transcript scra...
ellen,ellen degeneres for your approval transcript s...
gabriel,gabriel iglesias legend of fluffy transcript s...
hasan,scraps from the loft skip to content search mo...


In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
adam,sandler love scraps skip search moviesmovie re...
ali,ali baby cobra transcript scraps skip search m...
anthony,anthony jeselnik thoughts prayers transcript s...
bill,bill sorry way transcript scraps skip search m...
bo,scraps skip search moviesmovie reviewsmovie tr...
chad,chad daniels chaniels scraps skip search movie...
dave,chappelle age spin transcript scraps skip sear...
ellen,ellen degeneres approval transcript scraps ski...
gabriel,gabriel legend transcript scraps skip search m...
hasan,scraps skip search moviesmovie reviewsmovie tr...


In [ ]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
# Convert text.ENGLISH_STOP_WORDS to a list before performing the union operation
stop_words = list(text.ENGLISH_STOP_WORDS) + add_stop_words

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,aah,abc,abcs,ability,abortion,abortions,abraham,absurdities,abuse,abusedi,...,zippers,zipup,zombie,zombies,zone,zoneyoure,zoo,zookeeper,zoom,zuckerberg
adam,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
ali,0,1,0,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,0,0,0
anthony,0,0,0,0,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,0,0,1,0,0,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chad,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
ellen,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
gabriel,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.010*"christmas" + 0.004*"joke" + 0.004*"movie" + 0.003*"id" + 0.003*"look" + 0.003*"hell" + 0.002*"hes" + 0.002*"blah" + 0.002*"ghost" + 0.002*"chimp"'),
 (1,
  '0.010*"man" + 0.008*"cause" + 0.008*"day" + 0.008*"hes" + 0.007*"thing" + 0.007*"gon" + 0.007*"life" + 0.007*"shit" + 0.007*"guy" + 0.007*"way"')]

In [ ]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.007*"clinton" + 0.005*"cow" + 0.004*"mom" + 0.004*"hey" + 0.004*"way" + 0.004*"dad" + 0.004*"wife" + 0.004*"john" + 0.003*"day" + 0.003*"president"'),
 (1,
  '0.014*"man" + 0.010*"cause" + 0.007*"shit" + 0.007*"hes" + 0.007*"thing" + 0.007*"life" + 0.006*"guy" + 0.006*"gon" + 0.006*"way" + 0.006*"day"'),
 (2,
  '0.008*"day" + 0.007*"hes" + 0.007*"gon" + 0.007*"thing" + 0.007*"guy" + 0.006*"way" + 0.006*"life" + 0.006*"years" + 0.006*"shit" + 0.006*"man"')]

In [ ]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.008*"thing" + 0.008*"life" + 0.008*"gon" + 0.008*"shit" + 0.007*"day" + 0.007*"man" + 0.007*"things" + 0.006*"hes" + 0.006*"way" + 0.006*"years"'),
 (1,
  '0.017*"man" + 0.011*"hes" + 0.010*"shit" + 0.009*"fuck" + 0.007*"guy" + 0.006*"gon" + 0.006*"cause" + 0.006*"fucking" + 0.005*"life" + 0.005*"way"'),
 (2,
  '0.008*"cause" + 0.007*"way" + 0.007*"day" + 0.007*"guy" + 0.007*"thing" + 0.007*"gon" + 0.006*"life" + 0.006*"man" + 0.006*"hes" + 0.005*"house"'),
 (3,
  '0.011*"day" + 0.008*"women" + 0.008*"thing" + 0.007*"lot" + 0.007*"cause" + 0.006*"shit" + 0.005*"man" + 0.005*"fuck" + 0.005*"ass" + 0.005*"guns"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [ ]:
# Let's create a function to pull out nouns and adjectives from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)]
    return ' '.join(nouns_adj)

In [ ]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
adam,sandler love scraps loft skip search moviesmov...
ali,ali wong baby cobra transcript scraps loft ski...
anthony,anthony jeselnik thoughts prayers full transcr...
bill,bill im sorry way full transcript scraps loft ...
bo,scraps loft skip search moviesmovie reviewsmov...
chad,chad daniels chaniels scraps loft skip search ...
dave,chappelle age spin transcript scraps loft skip...
ellen,ellen degeneres approval transcript scraps lof...
gabriel,gabriel iglesias legend fluffy transcript scra...
hasan,scraps loft skip search moviesmovie reviewsmov...


In [ ]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aaaaah,aah,abc,abcs,ability,abject,able,ablebodied,abortion,abortions,...,zippers,zipup,zombie,zombies,zone,zoneyoure,zoo,zookeeper,zoom,zuckerberg
adam,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
ali,0,0,1,0,0,0,2,0,0,0,...,0,0,1,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
bill,1,0,0,1,0,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chad,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
ellen,0,0,0,0,1,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
gabriel,0,0,1,0,1,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.004*"christmas" + 0.004*"joke" + 0.004*"uh" + 0.003*"baby" + 0.003*"mom" + 0.003*"school" + 0.003*"point" + 0.002*"family" + 0.002*"mean" + 0.002*"fun"'),
 (1,
  '0.011*"shit" + 0.007*"fuck" + 0.005*"fucking" + 0.004*"dude" + 0.004*"uh" + 0.004*"son" + 0.004*"dick" + 0.003*"baby" + 0.003*"lets" + 0.003*"ass"')]

In [ ]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.007*"shit" + 0.006*"uh" + 0.006*"fuck" + 0.006*"fucking" + 0.004*"point" + 0.003*"son" + 0.003*"lets" + 0.003*"joke" + 0.003*"dude" + 0.003*"dick"'),
 (1,
  '0.007*"shit" + 0.005*"fck" + 0.004*"different" + 0.004*"men" + 0.004*"girl" + 0.004*"ass" + 0.004*"baby" + 0.004*"dude" + 0.003*"uh" + 0.003*"fuck"'),
 (2,
  '0.007*"shit" + 0.005*"christmas" + 0.004*"fuck" + 0.004*"mom" + 0.004*"baby" + 0.003*"school" + 0.003*"black" + 0.003*"wife" + 0.003*"gay" + 0.003*"family"')]

In [ ]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.010*"shit" + 0.005*"fck" + 0.004*"mom" + 0.004*"fuck" + 0.004*"school" + 0.004*"black" + 0.004*"joke" + 0.003*"water" + 0.003*"ahah" + 0.003*"different"'),
 (1,
  '0.008*"shit" + 0.007*"fucking" + 0.007*"fuck" + 0.006*"uh" + 0.004*"son" + 0.003*"lets" + 0.003*"point" + 0.003*"joke" + 0.003*"dick" + 0.003*"dude"'),
 (2,
  '0.005*"shit" + 0.004*"gay" + 0.004*"girl" + 0.004*"baby" + 0.004*"fuck" + 0.003*"somebody" + 0.003*"ladies" + 0.003*"boy" + 0.003*"bitch" + 0.003*"ellen"'),
 (3,
  '0.009*"christmas" + 0.007*"uh" + 0.005*"mom" + 0.005*"shit" + 0.004*"wife" + 0.004*"baby" + 0.003*"chinese" + 0.003*"doctor" + 0.003*"fun" + 0.003*"fuck"')]

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [ ]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.012*"shit" + 0.005*"fuck" + 0.005*"dude" + 0.004*"fck" + 0.003*"fucking" + 0.003*"point" + 0.003*"black" + 0.003*"wife" + 0.003*"different" + 0.003*"friend"'),
 (1,
  '0.006*"uh" + 0.006*"christmas" + 0.005*"mom" + 0.004*"joke" + 0.004*"family" + 0.003*"fun" + 0.003*"baby" + 0.003*"um" + 0.003*"wife" + 0.003*"mean"'),
 (2,
  '0.006*"joke" + 0.005*"son" + 0.005*"rape" + 0.004*"job" + 0.003*"nuts" + 0.003*"funny" + 0.003*"hair" + 0.003*"mm" + 0.003*"daughter" + 0.003*"jenner"'),
 (3,
  '0.010*"shit" + 0.009*"fuck" + 0.008*"fucking" + 0.006*"baby" + 0.005*"uh" + 0.005*"girl" + 0.005*"dick" + 0.004*"ass" + 0.004*"husband" + 0.004*"ladies"')]

These four topics look pretty decent. Let's settle on these for now.
* Topic 0: mom, parents
* Topic 1: husband, wife
* Topic 2: guns
* Topic 3: profanity

In [ ]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
# list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))
list(zip([a[0] if len(a) > 0 else -1 for a in corpus_transformed], data_dtmna.index))


[((2, 0.9995502), 'adam'),
 ((2, 0.99943465), 'ali'),
 ((2, 0.9993378), 'anthony'),
 ((3, 0.9995804), 'bill'),
 ((0, 0.8117326), 'bo'),
 ((3, 0.9995213), 'chad'),
 ((1, 0.9995283), 'dave'),
 ((3, 0.9994334), 'ellen'),
 ((2, 0.9996123), 'gabriel'),
 ((0, 0.8117325), 'hasan'),
 ((0, 0.9995037), 'jeff'),
 ((3, 0.9995631), 'jim'),
 ((2, 0.99956244), 'joe'),
 ((0, 0.99953467), 'john'),
 ((0, 0.9993587), 'louis'),
 ((0, 0.99930644), 'michelle'),
 ((2, 0.9995714), 'mike'),
 ((0, 0.9992648), 'nate'),
 ((3, 0.99960744), 'ricky'),
 ((0, 0.9995887), 'ronny'),
 ((2, 0.9995976), 'theo')]

For a first pass of LDA, these kind of make sense to me, so we'll call it a day for now.
* Topic 0: mom, parents [John, Hasan, Louis, Ronny]
* Topic 1: husband, wife [Dave]
* Topic 2: guns [Adam, Ali, Anthony]
* Topic 3: profanity [Bill, Jim, Ricky]

### Assignment:
1. Try further modifying the parameters of the topic models above and see if you can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.

Topic Modeling - Attempt #1                                                           
These topics aren't looking too great

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=5, passes=100)
lda.print_topics()

[(0,
  '0.006*"cause" + 0.006*"theyre" + 0.006*"want" + 0.005*"say" + 0.005*"oh" + 0.005*"going" + 0.005*"good" + 0.005*"didnt" + 0.005*"shit" + 0.005*"goes"'),
 (1,
  '0.007*"shit" + 0.006*"life" + 0.006*"cause" + 0.005*"ok" + 0.005*"hes" + 0.005*"thing" + 0.004*"lot" + 0.004*"theres" + 0.004*"theyre" + 0.004*"really"'),
 (2,
  '0.014*"fucking" + 0.008*"shit" + 0.008*"fuck" + 0.006*"goes" + 0.006*"love" + 0.006*"okay" + 0.006*"good" + 0.006*"uh" + 0.005*"man" + 0.005*"guys"'),
 (3,
  '0.014*"man" + 0.009*"little" + 0.006*"cause" + 0.005*"aint" + 0.005*"bro" + 0.005*"gotta" + 0.004*"ladies" + 0.004*"gay" + 0.004*"shit" + 0.004*"fuck"'),
 (4,
  '0.007*"say" + 0.005*"christmas" + 0.005*"didnt" + 0.005*"little" + 0.005*"joke" + 0.004*"did" + 0.004*"ive" + 0.004*"theyre" + 0.004*"mean" + 0.004*"day"')]

Topic Modeling - Attempt #2(nouns)

In [ ]:
ldan = models.LdaModel(corpus=corpusn, num_topics=6, id2word=id2wordn, passes=50)
ldan.print_topics()

[(0,
  '0.007*"christmas" + 0.007*"day" + 0.007*"thing" + 0.006*"way" + 0.006*"house" + 0.006*"years" + 0.006*"movie" + 0.005*"joke" + 0.005*"fluffy" + 0.005*"mom"'),
 (1,
  '0.010*"day" + 0.009*"thing" + 0.008*"hes" + 0.008*"gon" + 0.008*"guy" + 0.007*"man" + 0.007*"life" + 0.007*"shit" + 0.007*"fuck" + 0.006*"way"'),
 (2,
  '0.011*"life" + 0.008*"thing" + 0.007*"hes" + 0.007*"guy" + 0.007*"way" + 0.006*"kids" + 0.006*"day" + 0.006*"cause" + 0.005*"mom" + 0.005*"theyre"'),
 (3,
  '0.009*"shit" + 0.009*"lot" + 0.008*"husband" + 0.006*"women" + 0.006*"day" + 0.005*"cause" + 0.005*"dude" + 0.004*"god" + 0.004*"baby" + 0.004*"gon"'),
 (4,
  '0.021*"man" + 0.011*"cause" + 0.010*"shit" + 0.008*"gon" + 0.008*"hes" + 0.007*"way" + 0.006*"house" + 0.006*"lot" + 0.006*"life" + 0.006*"guy"'),
 (5,
  '0.009*"cause" + 0.008*"hes" + 0.006*"point" + 0.006*"night" + 0.006*"kind" + 0.006*"guy" + 0.005*"way" + 0.005*"car" + 0.005*"girl" + 0.005*"gon"')]

Topic Modeling - Attempt #3(nouns & adjectives)

In [ ]:
ldana = models.LdaModel(corpus=corpusna, num_topics=6, id2word=id2wordna, passes=100)
ldana.print_topics()

[(0,
  '0.006*"joke" + 0.006*"point" + 0.005*"school" + 0.005*"jenny" + 0.005*"baby" + 0.005*"anthony" + 0.004*"girl" + 0.004*"sure" + 0.004*"family" + 0.004*"mom"'),
 (1,
  '0.016*"christmas" + 0.005*"tit" + 0.005*"santa" + 0.005*"tiny" + 0.005*"course" + 0.004*"shit" + 0.004*"ha" + 0.003*"murder" + 0.003*"ok" + 0.003*"blah"'),
 (2,
  '0.006*"fck" + 0.006*"shit" + 0.005*"uh" + 0.005*"water" + 0.005*"mom" + 0.004*"job" + 0.004*"joke" + 0.004*"fun" + 0.003*"doctor" + 0.003*"door"'),
 (3,
  '0.006*"guns" + 0.006*"fucking" + 0.005*"business" + 0.005*"fuck" + 0.004*"ellen" + 0.004*"ass" + 0.004*"men" + 0.004*"um" + 0.004*"uh" + 0.004*"mean"'),
 (4,
  '0.000*"link" + 0.000*"faulty" + 0.000*"page" + 0.000*"fuck" + 0.000*"shit" + 0.000*"uh" + 0.000*"fucking" + 0.000*"joke" + 0.000*"baby" + 0.000*"door"'),
 (5,
  '0.012*"shit" + 0.008*"fuck" + 0.006*"fucking" + 0.005*"uh" + 0.005*"dude" + 0.005*"son" + 0.004*"dick" + 0.004*"lets" + 0.004*"black" + 0.003*"baby"')]

# Topic Modeling - Attempt #4(verbs)

In [ ]:
# Let's create a function to pull out verbs from a string of text
def verbs(text):
    '''Given a string of text, tokenize the text and pull out only the adjectives.'''
    is_verb = lambda pos:pos[:2] == 'VB'
    tokenized = word_tokenize(text)
    verbs = [word for (word, pos) in pos_tag(tokenized) if is_verb(pos)]
    return ' '.join(verbs)

In [ ]:
# Apply the verbs function to the transcripts to filter only on adjectives
data_verb = pd.DataFrame(data_clean.transcript.apply(verbs))
data_verb

,transcript
adam,transcript content carlindave chappelleintervi...
ali,content carlindave chappelleinterviewsplayboy ...
anthony,content carlindave chappelleinterviewsplayboy ...
bill,burr feel content carlindave chappelleintervie...
bo,content carlindave chappelleinterviewsplayboy ...
chad,dad transcript content carlindave chappelleint...
dave,dave content carlindave chappelleinterviewspla...
ellen,content carlindave chappelleinterviewsplayboy ...
gabriel,content carlindave chappelleinterviewsplayboy ...
hasan,content carlindave chappelleinterviewsplayboy ...


In [ ]:
# Create a new document-term matrix using only verbs, also remove common words with max_df
cvv= CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvv = cvv.fit_transform(data_verb.transcript)
data_dtmv = pd.DataFrame(data_cvv.toarray(), columns=cvv.get_feature_names_out())
data_dtmv.index = data_verb.index
data_dtmv

,aaah,abandoned,abbott,absorb,absorbing,abuse,abused,accepted,accomplished,according,...,youin,youll,younglooking,youno,younot,youoh,youve,zero,zipped,zoning
adam,0,0,1,0,0,0,0,0,2,0,...,0,2,1,1,0,1,1,0,0,0
ali,0,0,0,0,1,1,1,0,0,2,...,0,2,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
bill,0,0,0,1,0,0,0,0,0,0,...,0,2,0,0,0,0,1,0,0,1
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chad,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
dave,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,2,0,0,0
ellen,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
gabriel,0,0,0,0,0,0,0,1,0,1,...,0,1,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Create the gensim corpus
corpusv = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmv.transpose()))

# Create the vocabulary dictionary
id2wordv = dict((v, k) for k, v in cvv.vocabulary_.items())

In [ ]:
ldv = models.LdaModel(corpus=corpusv, num_topics=2, id2word=id2wordv, passes=50)
ldv.print_topics()

[(0,
  '0.026*"fucking" + 0.009*"shit" + 0.007*"fuck" + 0.006*"wan" + 0.005*"understand" + 0.004*"bring" + 0.004*"heard" + 0.004*"called" + 0.004*"hate" + 0.004*"die"'),
 (1,
  '0.007*"fcking" + 0.005*"told" + 0.005*"thinking" + 0.005*"knew" + 0.005*"die" + 0.005*"heard" + 0.004*"watching" + 0.004*"walked" + 0.004*"wasnt" + 0.004*"sitting"')]

In [ ]:
ldv = models.LdaModel(corpus=corpusv, num_topics=3, id2word=id2wordv, passes=50)
ldv.print_topics()

[(0,
  '0.009*"fcking" + 0.006*"thinking" + 0.006*"told" + 0.005*"die" + 0.005*"heard" + 0.005*"watching" + 0.005*"wasnt" + 0.005*"knew" + 0.004*"called" + 0.004*"sitting"'),
 (1,
  '0.034*"fucking" + 0.009*"shit" + 0.008*"fuck" + 0.005*"bring" + 0.005*"understand" + 0.005*"hed" + 0.005*"hit" + 0.005*"aint" + 0.004*"open" + 0.004*"okay"'),
 (2,
  '0.009*"walked" + 0.007*"wan" + 0.007*"fucking" + 0.006*"heard" + 0.005*"told" + 0.005*"shit" + 0.005*"met" + 0.004*"scared" + 0.004*"called" + 0.004*"fuck"')]

In [ ]:
ldv = models.LdaModel(corpus=corpusv, num_topics=4, id2word=id2wordv, passes=50)
ldv.print_topics()

[(0,
  '0.044*"fucking" + 0.010*"shit" + 0.007*"fuck" + 0.006*"bring" + 0.006*"understand" + 0.005*"sleep" + 0.005*"hit" + 0.004*"hate" + 0.004*"fucked" + 0.004*"sitting"'),
 (1,
  '0.008*"fucking" + 0.007*"shit" + 0.007*"wan" + 0.006*"aint" + 0.006*"walked" + 0.006*"fuck" + 0.005*"sitting" + 0.005*"told" + 0.005*"called" + 0.005*"heard"'),
 (2,
  '0.011*"fcking" + 0.009*"fucking" + 0.008*"hed" + 0.006*"die" + 0.006*"walked" + 0.006*"wasnt" + 0.006*"told" + 0.006*"heard" + 0.005*"knew" + 0.005*"wan"'),
 (3,
  '0.005*"thinking" + 0.005*"fucking" + 0.005*"uh" + 0.005*"knew" + 0.005*"die" + 0.005*"told" + 0.005*"fuck" + 0.004*"scared" + 0.004*"okay" + 0.004*"wan"')]

In [ ]:
ldv_best= models.LdaModel(corpus=corpusv, num_topics=5, id2word=id2wordv, passes=50)
ldv_best.print_topics()

[(0,
  '0.014*"fcking" + 0.010*"aint" + 0.009*"wan" + 0.007*"shit" + 0.006*"thinking" + 0.006*"wants" + 0.005*"play" + 0.005*"called" + 0.005*"okay" + 0.005*"beat"'),
 (1,
  '0.008*"told" + 0.008*"heard" + 0.006*"wasnt" + 0.005*"watching" + 0.005*"called" + 0.005*"felt" + 0.005*"knew" + 0.005*"scared" + 0.005*"hit" + 0.005*"thinking"'),
 (2,
  '0.008*"fucking" + 0.007*"uh" + 0.006*"hate" + 0.006*"use" + 0.006*"okay" + 0.006*"wan" + 0.005*"fuck" + 0.005*"die" + 0.005*"heard" + 0.005*"told"'),
 (3,
  '0.003*"searching" + 0.003*"pointing" + 0.003*"exist" + 0.000*"fucking" + 0.000*"shit" + 0.000*"wan" + 0.000*"hed" + 0.000*"fuck" + 0.000*"die" + 0.000*"knew"'),
 (4,
  '0.039*"fucking" + 0.009*"shit" + 0.009*"fuck" + 0.006*"walked" + 0.005*"hed" + 0.005*"bring" + 0.005*"sitting" + 0.005*"understand" + 0.005*"die" + 0.005*"knew"')]

In [ ]:
# Let's take a look at which topics each transcript contains
corpus_transformedv = ldv_best[corpusv]
list(zip([a[0] if len(a) > 0 else -1 for a in corpus_transformedv], data_dtmv.index))

[((4, 0.9988869), 'adam'),
 ((1, 0.9985155), 'ali'),
 ((2, 0.9983016), 'anthony'),
 ((4, 0.99910015), 'bill'),
 ((0, 0.050048616), 'bo'),
 ((2, 0.9988628), 'chad'),
 ((4, 0.9988241), 'dave'),
 ((1, 0.9986931), 'ellen'),
 ((1, 0.99916667), 'gabriel'),
 ((0, 0.050048612), 'hasan'),
 ((0, 0.998366), 'jeff'),
 ((4, 0.9989474), 'jim'),
 ((0, 0.99890554), 'joe'),
 ((4, 0.99870014), 'john'),
 ((4, 0.9982138), 'louis'),
 ((0, 0.99811125), 'michelle'),
 ((1, 0.9988196), 'mike'),
 ((2, 0.9981074), 'nate'),
 ((4, 0.9990766), 'ricky'),
 ((2, 0.99875814), 'ronny'),
 ((0, 0.99894834), 'theo')]

# Topic Modeling - Attempt #5(adverbs)

In [ ]:
# Let's create a function to pull out adverbs from a string of text
def adverbs(text):
    '''Given a string of text, tokenize the text and pull out only the adjectives.'''
    is_adverb = lambda pos:pos[:2] == 'RB'
    tokenized = word_tokenize(text)
    adverbs = [word for (word, pos) in pos_tag(tokenized) if is_adverb(pos)]
    return ' '.join(adverbs)

In [ ]:
# Apply the adverbs function to the transcripts to filter only on adverbs
data_adverb = pd.DataFrame(data_clean.transcript.apply(adverbs))
data_adverb

,transcript
adam,adam august rather oddly late back back yo ya ...
ali,ali so very very yes because now dont even jus...
anthony,too so much so here most politically not im no...
bill,right very much here here here here ridiculous...
bo,here maybe once directly
chad,too then just not then so right na even youve ...
dave,more very much here very im not ever detroit w...
ellen,less more instead poorly much instead meaningf...
gabriel,deeply even well i just na back back little to...
hasan,here maybe once directly


In [ ]:
# Create a new document-term matrix using only adverbs, also remove common words with max_df
cvav= CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvav = cvav.fit_transform(data_adverb.transcript)
data_dtmav = pd.DataFrame(data_cvav.toarray(), columns=cvav.get_feature_names_out())
data_dtmav.index = data_adverb.index
data_dtmav

,aback,abroad,abruptly,absolutely,accidentally,adam,admirably,admittedly,afrin,aggressively,...,youall,youd,youi,youll,youmoreali,youthere,youve,youvery,youyou,yuck
adam,0,0,0,2,1,3,0,0,0,0,...,0,0,1,3,0,1,0,0,0,0
ali,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,1,0,2,0,0,0
anthony,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
bill,0,0,0,3,2,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chad,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
dave,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
ellen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
gabriel,0,0,0,4,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Create the gensim corpus
corpusav = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmav.transpose()))

# Create the vocabulary dictionary
id2wordav = dict((v, k) for k, v in cvav.vocabulary_.items())

In [ ]:
ldav = models.LdaModel(corpus=corpusav, num_topics=2, id2word=id2wordav, passes=50)
ldav.print_topics()

[(0,
  '0.044*"na" + 0.036*"probably" + 0.026*"pretty" + 0.025*"ta" + 0.019*"better" + 0.019*"anymore" + 0.019*"long" + 0.017*"yes" + 0.015*"far" + 0.014*"ago"'),
 (1,
  '0.017*"soon" + 0.016*"twice" + 0.015*"completely" + 0.015*"anymore" + 0.011*"oh" + 0.010*"eventually" + 0.010*"probably" + 0.009*"ta" + 0.009*"straight" + 0.009*"suddenly"')]

In [ ]:
ldav = models.LdaModel(corpus=corpusav, num_topics=3, id2word=id2wordav, passes=50)
ldav.print_topics()

[(0,
  '0.041*"probably" + 0.032*"ta" + 0.031*"na" + 0.027*"pretty" + 0.021*"anymore" + 0.015*"long" + 0.014*"oh" + 0.014*"yes" + 0.013*"soon" + 0.013*"far"'),
 (1,
  '0.047*"na" + 0.029*"probably" + 0.021*"anymore" + 0.019*"long" + 0.019*"better" + 0.018*"ago" + 0.016*"ta" + 0.015*"far" + 0.015*"yes" + 0.013*"instead"'),
 (2,
  '0.030*"pretty" + 0.025*"better" + 0.018*"na" + 0.017*"ugly" + 0.017*"usually" + 0.012*"forever" + 0.011*"yes" + 0.009*"marnell" + 0.009*"smarter" + 0.009*"nice"')]

In [ ]:
ldav = models.LdaModel(corpus=corpusav, num_topics=4, id2word=id2wordav, passes=50)
ldav.print_topics()

[(0,
  '0.021*"pretty" + 0.021*"anymore" + 0.021*"soon" + 0.021*"probably" + 0.019*"completely" + 0.019*"twice" + 0.014*"eventually" + 0.014*"oh" + 0.012*"ta" + 0.010*"straight"'),
 (1,
  '0.064*"na" + 0.028*"probably" + 0.026*"anymore" + 0.020*"finally" + 0.015*"far" + 0.015*"immediately" + 0.015*"basically" + 0.015*"ago" + 0.015*"long" + 0.015*"better"'),
 (2,
  '0.025*"yes" + 0.020*"ago" + 0.019*"oh" + 0.018*"obviously" + 0.016*"long" + 0.015*"early" + 0.014*"suddenly" + 0.014*"better" + 0.013*"probably" + 0.013*"totally"'),
 (3,
  '0.043*"probably" + 0.038*"na" + 0.036*"ta" + 0.035*"pretty" + 0.021*"better" + 0.019*"long" + 0.017*"far" + 0.016*"yes" + 0.015*"exactly" + 0.014*"anymore"')]

# Findings
Nouns usually represent the core subjects or entities in a document — like "economy", "sports", "policy", etc.

Adjectives add context or description to nouns. They help you understand the tone or sentiment.

Verbs describe actions, but they’re often more functional than semantic in topic modeling.
Can be noisy: Verbs like "do", "make", "use" appear often but don’t help define distinct topics.

Adverbs are usually less informative.

